In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 23.3 MB/s eta 0:00:00


Make sure you upload the target, feature and index bladder numpy files

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, RobustScaler
import json
import optuna
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
import torch.nn.functional as F
import os

os.makedirs('/content/models', exist_ok=True)

model_type = 'bladder'

# Load data
features = np.load(f'/content/{model_type}_features.npy')
targets = np.load(f'/content/{model_type}_target.npy')
index = np.load(f'/content/{model_type}_index.npy')

class R2Loss(nn.Module):
    def __init__(self):
        super(R2Loss, self).__init__()

    def forward(self, pred, target):
        target_mean = torch.mean(target)
        ss_tot = torch.sum((target - target_mean) ** 2) + 1e-8
        ss_res = torch.sum((target - pred) ** 2)
        r2 = 1 - (ss_res / ss_tot)
        return -r2
class WeightedR2MSELoss(nn.Module):
    def __init__(self, r2_weight=0.8, mse_weight=0.2, r2_scaling=1.0, mse_scaling=1000000.0):
        super(WeightedR2MSELoss, self).__init__()
        self.r2_weight = r2_weight
        self.mse_weight = mse_weight
        self.r2_scaling = r2_scaling
        self.mse_scaling = mse_scaling

    def forward(self, pred, target):
        mse = torch.mean((pred - target) ** 2)
        target_mean = torch.mean(target)
        ss_tot = torch.sum((target - target_mean) ** 2) + 1e-8
        ss_res = torch.sum((target - pred) ** 2)
        r2 = 1 - (ss_res / ss_tot)
        return -self.r2_weight * self.r2_scaling * r2 + self.mse_weight * self.mse_scaling * mse

class TimeSeries(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, scale_y: bool = True, noise_level=0.0001):
        assert X.shape[0] == y.shape[0], "Mismatched number of samples"

        self.n, self.t, self.f = X.shape
        X = np.nan_to_num(X, nan=0.0, posinf=1e6, neginf=-1e6)
        self.x_scaler = RobustScaler(quantile_range=(5.0, 95.0))
        X_flat = X.reshape(-1, self.f)
        self.X_scaled = self.x_scaler.fit_transform(X_flat).reshape(self.n, self.t, self.f)
        self.X_scaled = np.clip(self.X_scaled, -10.0, 10.0)
        if noise_level > 0:
            self.X_scaled = self.X_scaled + np.random.normal(0, noise_level, self.X_scaled.shape)

        if np.isnan(self.X_scaled).any():
            print("Warning: NaNs detected in X_scaled after preprocessing")
            self.X_scaled = np.nan_to_num(self.X_scaled, nan=0.0)

        self.scale_y = scale_y
        if y.ndim == 1:
            y = y.reshape(-1, 1)

        y = np.nan_to_num(y, nan=0.0, posinf=1e6, neginf=-1e6)

        if scale_y:
            self.y_scaler = RobustScaler(quantile_range=(5.0, 95.0))
            self.y_scaled = self.y_scaler.fit_transform(y)
            self.y_scaled = np.clip(self.y_scaled, -10.0, 10.0)
        else:
            self.y_scaled = y
            self.y_scaler = None

        if np.isnan(self.y_scaled).any():
            print("Warning: NaNs detected in y_scaled after preprocessing")
            self.y_scaled = np.nan_to_num(self.y_scaled, nan=0.0)

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X_scaled[idx], dtype=torch.float32),
            torch.tensor(self.y_scaled[idx], dtype=torch.float32).view(-1)
        )

    def inverse_transform_y(self, y_tensor: torch.Tensor) -> np.ndarray:
        if self.scale_y and self.y_scaler is not None:
            return self.y_scaler.inverse_transform(y_tensor.detach().cpu().numpy())
        else:
            return y_tensor.detach().cpu().numpy()


class AttentionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2, bidirectional=False):
        super(AttentionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        dir_factor = 2 if bidirectional else 1
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional
        )
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * dir_factor, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1),
            nn.Softmax(dim=1)
        )
        self.fc1 = nn.Linear(hidden_size * dir_factor, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.bn2 = nn.BatchNorm1d(hidden_size // 2)
        self.fc3 = nn.Linear(hidden_size // 2, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        if torch.isnan(x).any():
            x = torch.nan_to_num(x, nan=0.0)
        lstm_out, _ = self.lstm(x)

        if torch.isnan(lstm_out).any():
            lstm_out = torch.nan_to_num(lstm_out, nan=0.0)

        attention_weights = self.attention(lstm_out)
        context = torch.sum(attention_weights * lstm_out, dim=1)

        out = self.fc1(context)
        if out.shape[0] > 1:
            out = self.bn1(out)
        out = F.gelu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        if out.shape[0] > 1:
            out = self.bn2(out)
        out = F.gelu(out)
        out = self.dropout(out)
        out = self.fc3(out)
        if torch.isnan(out).any():
            out = torch.nan_to_num(out, nan=0.0)

        return out


class EnsembleModel(nn.Module):
    def __init__(self, models, weights=None):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)
        if weights is None:
            self.weights = torch.ones(len(models)) / len(models)
        else:
            total = sum(weights)
            self.weights = torch.tensor([w/total for w in weights])

    def forward(self, x):
        outputs = []
        for i, model in enumerate(self.models):
            outputs.append(model(x))
        ensemble_output = torch.zeros_like(outputs[0])
        for i, output in enumerate(outputs):
            ensemble_output += self.weights[i] * output
        return ensemble_output
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [ ]:

def train(model, criterion, optimizer, train_dataloader, scheduler=None):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_y in train_dataloader:
        if torch.isnan(batch_X).any() or torch.isnan(batch_y).any():
            batch_X = torch.nan_to_num(batch_X, nan=0.0)
            batch_y = torch.nan_to_num(batch_y, nan=0.0)
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        epoch_loss += loss.item()

    if scheduler is not None:
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(epoch_loss / len(train_dataloader))
        else:
            scheduler.step()
    return epoch_loss / len(train_dataloader)


def validate(model, val_dataloader, dataset):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch_X, batch_y in val_dataloader:
            if torch.isnan(batch_X).any() or torch.isnan(batch_y).any():
                batch_X = torch.nan_to_num(batch_X, nan=0.0)
                batch_y = torch.nan_to_num(batch_y, nan=0.0)
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            if torch.isnan(outputs).any():
                outputs = torch.nan_to_num(outputs, nan=0.0)
            all_targets.append(batch_y.cpu())
            all_preds.append(outputs.cpu())
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    all_preds = torch.nan_to_num(all_preds, nan=0.0)
    all_targets = torch.nan_to_num(all_targets, nan=0.0)
    np_preds = all_preds.numpy()
    np_targets = all_targets.numpy()
    mse = mean_squared_error(np_targets, np_preds)
    r2 = r2_score(np_targets, np_preds)
    try:
        corr = np.corrcoef(np_targets.flatten(), np_preds.flatten())[0, 1]
    except:
        corr = 0.0
    y_pred = dataset.inverse_transform_y(all_preds)
    y_true = dataset.inverse_transform_y(all_targets)
    true_mse = mean_squared_error(y_true, y_pred)
    true_r2 = r2_score(y_true, y_pred)

    return mse, r2, corr, true_mse, true_r2


def objective(trial):
    hidden_size = trial.suggest_int("hidden_size", 64, 256, step=32)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    r2_weight = trial.suggest_float("r2_weight", 0.5, 0.95)
    mse_weight = 1.0 - r2_weight
    loss_type = trial.suggest_categorical("loss_type", ["r2", "weighted"])

    try:
        X_train, X_test, y_train, y_test = train_test_split(
            features,
            targets,
            test_size=0.2,
            shuffle=True,
            stratify=index[:, 1],
            random_state=42
        )
        train_data = TimeSeries(X_train, y_train, noise_level=0.0001)
        test_data = TimeSeries(X_test, y_test, noise_level=0)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)
        model = AttentionLSTM(
            input_size=features.shape[2],
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            output_size=1,
            bidirectional=bidirectional
        ).to(device)

        if loss_type == "r2":
            criterion = R2Loss()
        else:
            criterion = WeightedR2MSELoss(r2_weight=r2_weight, mse_weight=mse_weight)
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)
        epochs = 50
        best_r2 = -float('inf')
        patience = 7
        patience_counter = 0

        for epoch in range(epochs):
            train_loss = train(model, criterion, optimizer, train_loader, scheduler)
            _, r2, _, _, _ = validate(model, test_loader, test_data)
            if r2 > best_r2:
                best_r2 = r2
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break
        _, final_r2, _, true_mse, true_r2 = validate(model, test_loader, test_data)
        trial.report(true_r2, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        baseline_mse = 0.00000011
        baseline_r2 = 0.75


        if true_r2 < baseline_r2:
            return true_r2


        mse_factor = true_mse / baseline_mse
        if mse_factor > 10:
            penalty = 0.05 * (mse_factor / 10)
            return true_r2 - min(penalty, 0.1)

        return true_r2

    except Exception as e:
        print(f"Trial failed with error: {e}")
        return 0.0

study = optuna.create_study(direction="maximize",
                           pruner=optuna.pruners.MedianPruner(n_startup_trials=5))
study.optimize(objective, n_trials=50)

print("Best trial:")
print(f"R² Score: {study.best_trial.value}")
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")
with open(f'/content/models/{model_type}_r2_optimized_params.json', 'w') as file:
    json.dump(study.best_trial.params, file)
params = study.best_trial.params
hidden_size = params['hidden_size']
num_layers = params['num_layers']
dropout = params['dropout']
learning_rate = params['learning_rate']
batch_size = params['batch_size']
bidirectional = params['bidirectional']
weight_decay = params['weight_decay']
loss_type = params['loss_type']
if 'r2_weight' in params:
    r2_weight = params['r2_weight']
    mse_weight = 1.0 - r2_weight
else:
    r2_weight = 0.8
    mse_weight = 0.2
X_train, X_test, y_train, y_test, _, index_test = train_test_split(
    features,
    targets,
    index,
    test_size=0.2,
    shuffle=True,
    stratify=index[:, 1],
    random_state=42
)
train_data = TimeSeries(X_train, y_train, noise_level=0.0001)
test_data = TimeSeries(X_test, y_test, noise_level=0)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)
models = []
for i in range(3):
    h_size = int(hidden_size * (0.9 + 0.2 * np.random.random()))
    d_out = dropout * (0.8 + 0.4 * np.random.random())

    model = AttentionLSTM(
        input_size=features.shape[2],
        hidden_size=h_size,
        num_layers=num_layers,
        dropout=d_out,
        output_size=1,
        bidirectional=bidirectional
    ).to(device)

    models.append(model)
ensemble_model = EnsembleModel(models, weights=[0.5, 0.3, 0.2]).to(device)

if loss_type == "r2":
    criterion = R2Loss()
else:
    criterion = WeightedR2MSELoss(r2_weight=r2_weight, mse_weight=mse_weight)
optimizer = optim.AdamW(ensemble_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler1 = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=10, verbose=True, min_lr=1e-6)
scheduler2 = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=learning_rate/20)
num_epochs = 150
best_r2 = -float('inf')
best_model_state = None
patience = 25
patience_counter = 0

train_losses = []
val_mses = []
val_r2s = []
val_corrs = []

for epoch in tqdm(range(num_epochs)):
    train_loss = train(ensemble_model, criterion, optimizer, train_loader)
    scheduler2.step()
    mse, r2, corr, true_mse, true_r2 = validate(ensemble_model, test_loader, test_data)
    scheduler1.step(train_loss)
    train_losses.append(train_loss)
    val_mses.append(true_mse)
    val_r2s.append(true_r2)
    val_corrs.append(corr)
    if true_r2 > best_r2:
        best_r2 = true_r2
        best_model_state = {
            'ensemble': ensemble_model.state_dict(),
            'models': [model.state_dict() for model in models]
        }
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.7f}, "
              f"Val MSE: {true_mse:.8f}, Val R²: {true_r2:.7f}, Val Corr: {corr:.7f}")
if best_model_state:
    ensemble_model.load_state_dict(best_model_state['ensemble'])
    for i, model_state in enumerate(best_model_state['models']):
        models[i].load_state_dict(model_state)

plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(2, 2, 2)
plt.plot(val_mses)
plt.title('Validation MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')

plt.subplot(2, 2, 3)
plt.plot(val_r2s)
plt.title('Validation R²')
plt.xlabel('Epoch')
plt.ylabel('R²')

plt.subplot(2, 2, 4)
plt.plot(val_corrs)
plt.title('Validation Correlation')
plt.xlabel('Epoch')
plt.ylabel('Correlation')

plt.tight_layout()
plt.savefig(f'/content/models/{model_type}_r2_optimized_training_curves.png')
plt.close()

ensemble_model.eval()
predictions = []
targets = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = ensemble_model(X_batch).cpu()
        predictions.append(outputs)
        targets.append(y_batch)

y_pred_tensor = torch.cat(predictions)
y_true_tensor = torch.cat(targets)

y_pred = test_data.inverse_transform_y(y_pred_tensor)
y_true = test_data.inverse_transform_y(y_true_tensor)

df = pd.DataFrame(index_test, columns=["state", "year"])
df['year'] = df['year'].astype(int)
df["y_true"] = y_true.flatten()
df["y_pred"] = y_pred.flatten()

yg = df.sort_values('year').groupby('year')[['y_true', 'y_pred']].mean().reset_index()

final_mse = mean_squared_error(y_true, y_pred)
final_r2 = r2_score(y_true, y_pred)
final_corr = np.corrcoef(y_true.flatten(), y_pred.flatten())[0, 1]

print(f"Final Test MSE: {final_mse:.8f}")
print(f"Final Test R²: {final_r2:.7f}")
print(f"Final Test Correlation: {final_corr:.4f}")

fig = go.Figure()

fig.add_scatter(x=yg["year"], y=yg["y_pred"], mode="markers+lines", name="Predictions",
                marker=dict(symbol="x", size=10))
fig.add_scatter(x=yg["year"], y=yg["y_true"], mode="markers+lines", name="Ground Truth",
                marker=dict(size=8))

yearly_r2 = r2_score(yg["y_true"], yg["y_pred"])
yearly_corr = np.corrcoef(yg["y_true"], yg["y_pred"])[0, 1]

fig.update_layout(
    title=f"{model_type.capitalize()} Cancer Model (R² Optimized) - Yearly Average (R² = {yearly_r2:.3f}, Corr = {yearly_corr:.3f})",
    xaxis_title="Year",
    yaxis_title="Value",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.write_html(f'/content/models/{model_type}_r2_optimized_yearly_predictions.html')

fig2 = go.Figure()

fig2.add_scatter(x=y_true.flatten(), y=y_pred.flatten(), mode="markers",
                marker=dict(size=8, opacity=0.6))

max_val = max(np.max(y_true), np.max(y_pred))
min_val = min(np.min(y_true), np.min(y_pred))
fig2.add_scatter(x=[min_val, max_val], y=[min_val, max_val], mode="lines",
                name="Perfect Prediction", line=dict(dash="dash", color="gray"))

fig2.update_layout(
    title=f"{model_type.capitalize()} Cancer Model (R² Optimized) - Predictions vs Ground Truth (R² = {final_r2:.7f}, MSE = {final_mse:.8f})",
    xaxis_title="Ground Truth",
    yaxis_title="Prediction",
    template="plotly_white"
)

fig2.write_html(f'/content/models/{model_type}_r2_optimized_scatter_predictions.html')

torch.save({
    'ensemble': ensemble_model.state_dict(),
    'models': [model.state_dict() for model in models],
    'params': params
}, f"/content/models/{model_type}_r2_optimized_model.pth")

# ----------------
# Two-Stage Training Approach
# ----------------

print("\nStarting two-stage training approach...")
base_model = AttentionLSTM(
    input_size=features.shape[2],
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    output_size=1,
    bidirectional=bidirectional
).to(device)
stage1_criterion = nn.MSELoss()
stage1_optimizer = optim.Adam(base_model.parameters(), lr=learning_rate, weight_decay=weight_decay/10)
stage1_scheduler = ReduceLROnPlateau(stage1_optimizer, mode='min', factor=0.5, patience=5)
print("Stage 1: MSE Optimization")
stage1_epochs = 50
best_mse = float('inf')
best_stage1_state = None
patience_counter = 0

for epoch in range(stage1_epochs):
    train_loss = train(base_model, stage1_criterion, stage1_optimizer, train_loader, stage1_scheduler)
    mse, r2, _, _, _ = validate(base_model, test_loader, test_data)
    if mse < best_mse:
        best_mse = mse
        best_stage1_state = base_model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= 10:
            print(f"Early stopping stage 1 at epoch {epoch+1}")
            break

    if (epoch + 1) % 10 == 0:
        print(f"Stage 1 - Epoch {epoch+1}/{stage1_epochs}, Train Loss: {train_loss:.7f}, Val MSE: {mse:.8f}, Val R²: {r2:.7f}")
base_model.load_state_dict(best_stage1_state)
print("\nStage 2: R² Optimization")
for name, param in base_model.named_parameters():
    if 'lstm' in name or 'fc1' in name:
        param.requires_grad = False

stage2_criterion = R2Loss()
stage2_optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, base_model.parameters()),
    lr=learning_rate / 5,
    weight_decay=weight_decay / 2
)
stage2_scheduler = CosineAnnealingWarmRestarts(stage2_optimizer, T_0=5, T_mult=2, eta_min=1e-6)

stage2_epochs = 50
best_r2 = -float('inf')
best_stage2_state = None
patience_counter = 0

for epoch in range(stage2_epochs):
    train_loss = train(base_model, stage2_criterion, stage2_optimizer, train_loader, stage2_scheduler)
    _, r2, _, true_mse, true_r2 = validate(base_model, test_loader, test_data)

    if true_r2 > best_r2:
        best_r2 = true_r2
        best_stage2_state = base_model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= 15:
            print(f"Early stopping stage 2 at epoch {epoch+1}")
            break

    if (epoch + 1) % 5 == 0:
        print(f"Stage 2 - Epoch {epoch+1}/{stage2_epochs}, Train Loss: {train_loss:.7f}, Val R²: {true_r2:.7f}, Val MSE: {true_mse:.8f}")

base_model.load_state_dict(best_stage2_state)

_, _, _, two_stage_mse, two_stage_r2 = validate(base_model, test_loader, test_data)

print("\nTwo-Stage Model Results:")
print(f"MSE: {two_stage_mse:.8f}")
print(f"R²: {two_stage_r2:.7f}")

torch.save(base_model.state_dict(), f"/content/models/{model_type}_two_stage_model.pth")

# ----------------
# Trend-focused Model
# ----------------

print("\nTraining trend-focused model...")
class TrendDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, scale_y: bool = True):
        assert X.shape[0] == y.shape[0], "Mismatched number of samples"
        self.original_X = X
        self.original_y = y.reshape(-1, 1) if y.ndim == 1 else y
        self.x_scaler = RobustScaler(quantile_range=(5.0, 95.0))
        X_flat = np.nan_to_num(X.reshape(-1, X.shape[2]), nan=0.0)
        X_scaled = self.x_scaler.fit_transform(X_flat).reshape(X.shape)
        X_scaled = np.clip(X_scaled, -10.0, 10.0)
        self.n, self.t, self.f = X.shape
        self.features = []
        self.targets = []
        self.indices = []
        for i in range(self.n):
            x_sample = X_scaled[i]
            y_sample = y[i]
            if np.isnan(x_sample).any() or np.isnan(y_sample):
                continue
            if self.t > 1:
                x_pct_change = np.zeros((self.t-1, self.f))
                for j in range(self.t-1):
                    denom = np.abs(x_sample[j]) + 1e-8
                    x_pct_change[j] = (x_sample[j+1] - x_sample[j]) / denom
                x_pct_change = np.clip(x_pct_change, -10.0, 10.0)
                x_combined = np.column_stack([
                    x_pct_change.reshape(-1),
                    x_sample[:-1].reshape(-1)
                ])

                self.features.append(x_combined)
                self.targets.append(y_sample)
                self.indices.append(i)

        if len(self.features) > 0:
            self.features = np.array(self.features)
            self.targets = np.array(self.targets).reshape(-1, 1)
            self.indices = np.array(self.indices)
            self.scale_y = scale_y
            if scale_y:
                self.y_scaler = RobustScaler(quantile_range=(5.0, 95.0))
                self.targets_scaled = self.y_scaler.fit_transform(self.targets)
                self.targets_scaled = np.clip(self.targets_scaled, -10.0, 10.0)
            else:
                self.targets_scaled = self.targets
                self.y_scaler = None
        else:
            print("Warning: No valid samples found for trend dataset")
            self.features = np.zeros((1, 1, self.f * 2))
            self.targets = np.zeros((1, 1))
            self.targets_scaled = np.zeros((1, 1))
            self.indices = np.zeros(1)
            self.scale_y = scale_y
            self.y_scaler = None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.features[idx], dtype=torch.float32),
            torch.tensor(self.targets_scaled[idx], dtype=torch.float32).view(-1)
        )

    def inverse_transform_y(self, y_tensor: torch.Tensor) -> np.ndarray:
        if self.scale_y and self.y_scaler is not None:
            return self.y_scaler.inverse_transform(y_tensor.detach().cpu().numpy())
        else:
            return y_tensor.detach().cpu().numpy()
class TrendModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
        super(TrendModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)

        self.fc3 = nn.Linear(hidden_size, hidden_size // 2)
        self.bn3 = nn.BatchNorm1d(hidden_size // 2)

        self.fc4 = nn.Linear(hidden_size // 2, output_size)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size = x.size(0)
        if torch.isnan(x).any():
            x = torch.nan_to_num(x, nan=0.0)
        if x.dim() > 2:
            x = x.reshape(batch_size, -1)
        x = self.fc1(x)
        if batch_size > 1:
            x = self.bn1(x)
        x = F.gelu(x)
        x = self.dropout(x)
        identity = x
        x = self.fc2(x)
        if batch_size > 1:
            x = self.bn2(x)
        x = F.gelu(x)
        x = self.dropout(x)
        x = x + identity
        x = self.fc3(x)
        if batch_size > 1:
            x = self.bn3(x)
        x = F.gelu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        if torch.isnan(x).any():
            x = torch.nan_to_num(x, nan=0.0)

        return x
trend_train_data = TrendDataset(X_train, y_train)
trend_test_data = TrendDataset(X_test, y_test)
input_dim = trend_train_data.features.shape[1] * trend_train_data.features.shape[2]
trend_train_loader = DataLoader(trend_train_data, batch_size=32, shuffle=True, num_workers=2)
trend_test_loader = DataLoader(trend_test_data, batch_size=32, shuffle=False, num_workers=2)
trend_model = TrendModel(
    input_size=input_dim,
    hidden_size=128,
    output_size=1,
    dropout=0.2
).to(device)
trend_criterion = R2Loss()
trend_optimizer = optim.AdamW(trend_model.parameters(), lr=0.001, weight_decay=1e-5)
trend_scheduler = ReduceLROnPlateau(trend_optimizer, mode='min', factor=0.5, patience=10, verbose=True)
trend_epochs = 100
best_trend_r2 = -float('inf')
best_trend_state = None
patience = 20
patience_counter = 0
print("\nTraining trend-focused model...")
for epoch in range(trend_epochs):
    train_loss = train(trend_model, trend_criterion, trend_optimizer, trend_train_loader, trend_scheduler)
    _, r2, _, true_mse, true_r2 = validate(trend_model, trend_test_loader, trend_test_data)

    if true_r2 > best_trend_r2:
        best_trend_r2 = true_r2
        best_trend_state = trend_model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping trend model at epoch {epoch+1}")
            break
    if (epoch + 1) % 10 == 0:
        print(f"Trend Model - Epoch {epoch+1}/{trend_epochs}, Train Loss: {train_loss:.7f}, "
              f"Val R²: {true_r2:.7f}, Val MSE: {true_mse:.8f}")

trend_model.load_state_dict(best_trend_state)
_, _, _, trend_mse, trend_r2 = validate(trend_model, trend_test_loader, trend_test_data)

print("\nTrend Model Results:")
print(f"MSE: {trend_mse:.8f}")
print(f"R²: {trend_r2:.7f}")

torch.save(trend_model.state_dict(), f"/content/models/{model_type}_trend_model.pth")

# ----------------
# Ultra Ensemble - Final Solution
# ----------------

print("\nCreating ultra ensemble solution...")
def ultra_ensemble_predict(X_batch, weights=None):
    """
    Combines predictions from all three model approaches:
    1. Ensemble LSTM model
    2. Two-stage model
    3. Trend model

    Returns weighted average of predictions
    """
    if weights is None:
        weights = [0.5, 0.3, 0.2]
    X_batch = X_batch.to(device)
    ensemble_model.eval()
    with torch.no_grad():
        ensemble_pred = ensemble_model(X_batch).cpu()
    base_model.eval()
    with torch.no_grad():
        two_stage_pred = base_model(X_batch).cpu()
    X_np = X_batch.cpu().numpy()
    n, t, f = X_np.shape
    trend_features = []

    for i in range(n):
        x_sample = X_np[i]
        if t > 1:
            x_pct_change = np.zeros((t-1, f))
            for j in range(t-1):
                denom = np.abs(x_sample[j]) + 1e-8
                x_pct_change[j] = (x_sample[j+1] - x_sample[j]) / denom
            x_pct_change = np.clip(x_pct_change, -10.0, 10.0)
            x_combined = np.column_stack([
                x_pct_change.reshape(-1),
                x_sample[:-1].reshape(-1)
            ])

            trend_features.append(x_combined)
    if trend_features:
        trend_features = torch.tensor(np.array(trend_features), dtype=torch.float32).to(device)
        trend_model.eval()
        with torch.no_grad():
            trend_pred = trend_model(trend_features).cpu()
    else:
        trend_pred = (ensemble_pred + two_stage_pred) / 2
    final_pred = (
        weights[0] * ensemble_pred +
        weights[1] * two_stage_pred +
        weights[2] * trend_pred
    )

    return final_pred

weight_options = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
best_weights = None
best_ultra_r2 = -float('inf')

print("Optimizing ultra ensemble weights...")
for w1 in weight_options:
    for w2 in weight_options:
        for w3 in weight_options:
            if abs(w1 + w2 + w3 - 1.0) > 1e-6:
                continue

            weights = [w1, w2, w3]

            all_preds = []
            all_targets = []

            for X_batch, y_batch in test_loader:
                pred = ultra_ensemble_predict(X_batch, weights)
                all_preds.append(pred)
                all_targets.append(y_batch)

            y_pred_tensor = torch.cat(all_preds)
            y_true_tensor = torch.cat(all_targets)

            y_pred = test_data.inverse_transform_y(y_pred_tensor)
            y_true = test_data.inverse_transform_y(y_true_tensor)

            ultra_r2 = r2_score(y_true, y_pred)

            if ultra_r2 > best_ultra_r2:
                best_ultra_r2 = ultra_r2
                best_weights = weights

                ultra_mse = mean_squared_error(y_true, y_pred)

                print(f"New best weights: {weights} - R²: {ultra_r2:.7f}, MSE: {ultra_mse:.8f}")

print(f"\nFinal ultra ensemble with weights: {best_weights}")

all_preds = []
all_targets = []

for X_batch, y_batch in test_loader:
    pred = ultra_ensemble_predict(X_batch, best_weights)
    all_preds.append(pred)
    all_targets.append(y_batch)

y_pred_tensor = torch.cat(all_preds)
y_true_tensor = torch.cat(all_targets)

y_pred = test_data.inverse_transform_y(y_pred_tensor)
y_true = test_data.inverse_transform_y(y_true_tensor)

ultra_mse = mean_squared_error(y_true, y_pred)
ultra_r2 = r2_score(y_true, y_pred)
ultra_corr = np.corrcoef(y_true.flatten(), y_pred.flatten())[0, 1]

print("\nUltra Ensemble Final Results:")
print(f"MSE: {ultra_mse:.8f}")
print(f"R²: {ultra_r2:.7f}")
print(f"Correlation: {ultra_corr:.4f}")

df_ultra = pd.DataFrame(index_test, columns=["state", "year"])
df_ultra['year'] = df_ultra['year'].astype(int)
df_ultra["y_true"] = y_true.flatten()
df_ultra["y_pred"] = y_pred.flatten()
yg_ultra = df_ultra.sort_values('year').groupby('year')[['y_true', 'y_pred']].mean().reset_index()
yearly_ultra_r2 = r2_score(yg_ultra["y_true"], yg_ultra["y_pred"])
yearly_ultra_corr = np.corrcoef(yg_ultra["y_true"], yg_ultra["y_pred"])[0, 1]
fig_ultra = go.Figure()

fig_ultra.add_scatter(x=yg_ultra["year"], y=yg_ultra["y_pred"], mode="markers+lines", name="Predictions",
                marker=dict(symbol="x", size=10))
fig_ultra.add_scatter(x=yg_ultra["year"], y=yg_ultra["y_true"], mode="markers+lines", name="Ground Truth",
                marker=dict(size=8))

fig_ultra.update_layout(
    title=f"{model_type.capitalize()} Cancer Model (Ultra Ensemble) - Yearly Average (R² = {yearly_ultra_r2:.3f}, Corr = {yearly_ultra_corr:.3f})",
    xaxis_title="Year",
    yaxis_title="Value",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig_ultra.write_html(f'/content/models/{model_type}_ultra_ensemble_yearly_predictions.html')
fig_ultra2 = go.Figure()

fig_ultra2.add_scatter(x=y_true.flatten(), y=y_pred.flatten(), mode="markers",
                marker=dict(size=8, opacity=0.6))

max_val = max(np.max(y_true), np.max(y_pred))
min_val = min(np.min(y_true), np.min(y_pred))
fig_ultra2.add_scatter(x=[min_val, max_val], y=[min_val, max_val], mode="lines",
                name="Perfect Prediction", line=dict(dash="dash", color="gray"))

fig_ultra2.update_layout(
    title=f"{model_type.capitalize()} Cancer Model (Ultra Ensemble) - Predictions vs Ground Truth (R² = {ultra_r2:.7f}, MSE: {ultra_mse:.8f})",
    xaxis_title="Ground Truth",
    yaxis_title="Prediction",
    template="plotly_white"
)

fig_ultra2.write_html(f'/content/models/{model_type}_ultra_ensemble_scatter_predictions.html')
torch.save({
    'ensemble_model': ensemble_model.state_dict(),
    'two_stage_model': base_model.state_dict(),
    'trend_model': trend_model.state_dict(),
    'weights': best_weights,
    'params': params
}, f"/content/models/{model_type}_ultra_ensemble_model.pth")


print("\n=== FINAL MODEL COMPARISON ===")
print(f"Ensemble model:     R² = {final_r2:.7f}, MSE = {final_mse:.8f}")
print(f"Two-stage model:    R² = {two_stage_r2:.7f}, MSE = {two_stage_mse:.8f}")
print(f"Trend model:        R² = {trend_r2:.7f}, MSE = {trend_mse:.8f}")
print(f"Ultra ensemble:     R² = {ultra_r2:.7f}, MSE = {ultra_mse:.8f}")
print("============================")

[I 2025-05-12 21:20:11,761] A new study created in memory with name: no-name-82ca9734-0a5c-4417-bbb5-4aba8b2f616d
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-05-12 21:20:39,188] Trial 0 finished with value: 0.4226493239402771 and parameters: {'hidden_size': 128, 'num_layers': 3, 'dropout': 0.46793601865559203, 'learning_rate': 0.0008785847239827912, 'batch_size': 8, 'bidirectional': True, 'weight_decay': 3.5253726320804834e-06, 'r2_weight': 0.6287221417713786, 'loss_type': 'r2'}. Best is trial 0 with value: 0.4226493239402771.
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-05-12 21:20:46,095] Trial 1 finished with value: 0.357448935508728 and parameters: {'hidden_size': 128, 'num_

Trial failed with error: 


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-05-12 21:24:18,616] Trial 16 finished with value: 0.4979878067970276 and parameters: {'hidden_size': 224, 'num_layers': 2, 'dropout': 0.4310468370116307, 'learning_rate': 0.0005117979565484107, 'batch_size': 64, 'bidirectional': False, 'weight_decay': 0.00019797683113151066, 'r2_weight': 0.8617371028495968, 'loss_type': 'weighted'}. Best is trial 8 with value: 0.5971581935882568.
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-05-12 21:24:23,531] Trial 17 finished with value: 0.48867732286453247 and parameters: {'hidden_size': 160, 'num_layers': 1, 'dropout': 0.3350041972041576, 'learning_rate': 0.006607323782761405, 'batch_size': 64, 'bi

Trial failed with error: 


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-05-12 21:24:58,847] Trial 22 finished with value: 0.2775992751121521 and parameters: {'hidden_size': 256, 'num_layers': 2, 'dropout': 0.27095024735137674, 'learning_rate': 0.0014782030935480712, 'batch_size': 64, 'bidirectional': False, 'weight_decay': 7.052884897232424e-06, 'r2_weight': 0.5625026758218589, 'loss_type': 'r2'}. Best is trial 8 with value: 0.5971581935882568.
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-05-12 21:25:01,682] Trial 23 finished with value: 0.0 and parameters: {'hidden_size': 224, 'num_layers': 3, 'dropout': 0.3493185291062939, 'learning_rate': 0.003814724461795439, 'batch_size': 64, 'bidirectional': False, '

...

 49%|████▊     | 73/150 [01:26<01:31,  1.19s/it]

Early stopping at epoch 74


Final Test MSE: 0.00000011
Final Test R²: 0.6005586
Final Test Correlation: 0.7821

Starting two-stage training approach...
Stage 1: MSE Optimization
Stage 1 - Epoch 10/50, Train Loss: 0.0893499, Val MSE: 0.08112255, Val R²: 0.3287996
Stage 1 - Epoch 20/50, Train Loss: 0.0806374, Val MSE: 0.08532406, Val R²: 0.2940367
Stage 1 - Epoch 30/50, Train Loss: 0.0522640, Val MSE: 0.07234387, Val R²: 0.4014336
Stage 1 - Epoch 40/50, Train Loss: 0.0517519, Val MSE: 0.05842129, Val R²: 0.5166277
Stage 1 - Epoch 50/50, Train Loss: 0.0533778, Val MSE: 0.07524016, Val R²: 0.3774699

Stage 2: R² Optimization
Stage 2 - Epoch 5/50, Train Loss: -0.3732480, Val R²: 0.4180571, Val MSE: 0.00000017
Stage 2 - Epoch 10/50, Train Loss: -0.2204035, Val R²: 0.4449029, Val MSE: 0.00000016
Stage 2 - Epoch 15/50, Train Loss: -0.3720224, Val R²: 0.4472895, Val MSE: 0.00000016
Early stopping stage 2 at epoch 17

Two-Stage Model Results:
MSE: 0.00000018
R²: 0.3753374

Training trend-focused model...

Training trend-fo

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning:

The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.



Trend Model - Epoch 10/100, Train Loss: -0.5856354, Val R²: 0.3570626, Val MSE: 0.00000018
Trend Model - Epoch 20/100, Train Loss: -0.5482164, Val R²: 0.4144678, Val MSE: 0.00000017
Trend Model - Epoch 30/100, Train Loss: -0.7684197, Val R²: 0.4308795, Val MSE: 0.00000016
Trend Model - Epoch 40/100, Train Loss: -0.5608040, Val R²: 0.4680851, Val MSE: 0.00000015
Trend Model - Epoch 50/100, Train Loss: -0.7833325, Val R²: 0.4138120, Val MSE: 0.00000017
Early stopping trend model at epoch 60

Trend Model Results:
MSE: 0.00000017
R²: 0.4119355

Creating ultra ensemble solution...
Optimizing ultra ensemble weights...
New best weights: [0.0, 0.0, 1.0] - R²: 0.4119355, MSE: 0.00000017
New best weights: [0.0, 0.2, 0.8] - R²: 0.5223169, MSE: 0.00000014
New best weights: [0.0, 0.4, 0.6] - R²: 0.5738478, MSE: 0.00000012
New best weights: [0.2, 0.2, 0.6] - R²: 0.6075628, MSE: 0.00000011
New best weights: [0.2, 0.4, 0.4] - R²: 0.6177873, MSE: 0.00000011
New best weights: [0.4, 0.0, 0.6] - R²: 0.620

REPRODUCE

In [ ]:
import os
import torch
import numpy as np
import random
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import RobustScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.model_selection import train_test_split
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_type = "bladder"

os.makedirs('/content/models', exist_ok=True)

def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

class R2Loss(nn.Module):
    def __init__(self, eps=1e-6):
        super(R2Loss, self).__init__()
        self.eps = eps

    def forward(self, y_pred, y_true):
        if torch.isnan(y_pred).any() or torch.isnan(y_true).any():
            y_pred = torch.nan_to_num(y_pred, nan=0.0)
            y_true = torch.nan_to_num(y_true, nan=0.0)

        ss_tot = torch.sum((y_true - torch.mean(y_true)) ** 2) + self.eps
        ss_res = torch.sum((y_true - y_pred) ** 2) + self.eps
        r2 = 1 - ss_res / ss_tot
        return -r2

class WeightedR2MSELoss(nn.Module):
    def __init__(self, r2_weight=0.8, mse_weight=0.2, eps=1e-6):
        super(WeightedR2MSELoss, self).__init__()
        self.r2_weight = r2_weight
        self.mse_weight = mse_weight
        self.r2_loss = R2Loss(eps=eps)
        self.mse_loss = nn.MSELoss()

    def forward(self, y_pred, y_true):
        if torch.isnan(y_pred).any() or torch.isnan(y_true).any():
            y_pred = torch.nan_to_num(y_pred, nan=0.0)
            y_true = torch.nan_to_num(y_true, nan=0.0)

        r2 = self.r2_loss(y_pred, y_true)
        mse = self.mse_loss(y_pred, y_true)
        return self.r2_weight * r2 + self.mse_weight * mse

class AttentionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, output_size=1, bidirectional=False):
        super(AttentionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional
        )
        self.dropout = nn.Dropout(dropout)
        self.attention = nn.Linear(hidden_size * (2 if bidirectional else 1), 1)
        self.fc1 = nn.Linear(hidden_size * (2 if bidirectional else 1), hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        if torch.isnan(x).any():
            x = torch.nan_to_num(x, nan=0.0)
        batch_size, seq_len, features = x.size()

        outputs, _ = self.lstm(x)

        attention_weights = F.softmax(self.attention(outputs), dim=1)
        context_vector = torch.sum(attention_weights * outputs, dim=1)

        x = F.gelu(self.fc1(context_vector))
        x = self.dropout(x)
        x = self.fc2(x)

        if torch.isnan(x).any():
            x = torch.nan_to_num(x, nan=0.0)

        return x

class EnsembleModel(nn.Module):
    def __init__(self, models, weights=None):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)
        if weights is None:
            self.weights = nn.Parameter(torch.ones(len(models)) / len(models))
        else:
            self.weights = nn.Parameter(torch.tensor(weights, dtype=torch.float32))

    def forward(self, x):
        if torch.isnan(x).any():
            x = torch.nan_to_num(x, nan=0.0)

        outputs = []
        for model in self.models:
            output = model(x)
            outputs.append(output)

        stacked_outputs = torch.stack(outputs, dim=-1)

        weights = F.softmax(self.weights, dim=0)

        weighted_sum = torch.sum(stacked_outputs * weights, dim=-1)

        if torch.isnan(weighted_sum).any():
            weighted_sum = torch.nan_to_num(weighted_sum, nan=0.0)

        return weighted_sum

class TrendModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
        super(TrendModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)

        self.fc3 = nn.Linear(hidden_size, hidden_size // 2)
        self.bn3 = nn.BatchNorm1d(hidden_size // 2)

        self.fc4 = nn.Linear(hidden_size // 2, output_size)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size = x.size(0)
        if torch.isnan(x).any():
            x = torch.nan_to_num(x, nan=0.0)
        if x.dim() > 2:
            x = x.reshape(batch_size, -1)
        x = self.fc1(x)
        if batch_size > 1:
            x = self.bn1(x)
        x = F.gelu(x)
        x = self.dropout(x)
        identity = x
        x = self.fc2(x)
        if batch_size > 1:
            x = self.bn2(x)
        x = F.gelu(x)
        x = self.dropout(x)
        x = x + identity
        x = self.fc3(x)
        if batch_size > 1:
            x = self.bn3(x)
        x = F.gelu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        if torch.isnan(x).any():
            x = torch.nan_to_num(x, nan=0.0)

        return x

class TimeSeries(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, noise_level=0.0):
        self.original_X = X.copy()
        self.original_y = y.reshape(-1, 1) if y.ndim == 1 else y.copy()

        self.x_scaler = RobustScaler(quantile_range=(5.0, 95.0))
        X_flat = np.nan_to_num(X.reshape(-1, X.shape[2]), nan=0.0)
        self.X_scaled = self.x_scaler.fit_transform(X_flat).reshape(X.shape)
        self.X_scaled = np.clip(self.X_scaled, -10.0, 10.0)

        self.y_scaler = RobustScaler(quantile_range=(5.0, 95.0))
        self.y_scaled = self.y_scaler.fit_transform(self.original_y)
        self.y_scaled = np.clip(self.y_scaled, -10.0, 10.0)

        self.noise_level = noise_level

    def __len__(self):
        return len(self.X_scaled)

    def __getitem__(self, idx):
        X_sample = self.X_scaled[idx].astype(np.float32)
        y_sample = self.y_scaled[idx].astype(np.float32)

        if self.noise_level > 0:
            noise = np.random.normal(0, self.noise_level, X_sample.shape)
            X_sample = X_sample + noise

        return torch.tensor(X_sample, dtype=torch.float32), torch.tensor(y_sample, dtype=torch.float32).view(-1)

    def inverse_transform_y(self, y_tensor):
        if hasattr(y_tensor, "detach"):
            y_numpy = y_tensor.detach().cpu().numpy()
        else:
            y_numpy = y_tensor
        return self.y_scaler.inverse_transform(y_numpy)

class TrendDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, scale_y: bool = True):
        assert X.shape[0] == y.shape[0], "Mismatched number of samples"
        self.original_X = X
        self.original_y = y.reshape(-1, 1) if y.ndim == 1 else y
        self.x_scaler = RobustScaler(quantile_range=(5.0, 95.0))
        X_flat = np.nan_to_num(X.reshape(-1, X.shape[2]), nan=0.0)
        X_scaled = self.x_scaler.fit_transform(X_flat).reshape(X.shape)
        X_scaled = np.clip(X_scaled, -10.0, 10.0)
        self.n, self.t, self.f = X.shape
        self.features = []
        self.targets = []
        self.indices = []
        for i in range(self.n):
            x_sample = X_scaled[i]
            y_sample = y[i]
            if np.isnan(x_sample).any() or np.isnan(y_sample):
                continue
            if self.t > 1:
                x_pct_change = np.zeros((self.t-1, self.f))
                for j in range(self.t-1):
                    denom = np.abs(x_sample[j]) + 1e-8
                    x_pct_change[j] = (x_sample[j+1] - x_sample[j]) / denom
                x_pct_change = np.clip(x_pct_change, -10.0, 10.0)
                x_combined = np.column_stack([
                    x_pct_change.reshape(-1),
                    x_sample[:-1].reshape(-1)
                ])

                self.features.append(x_combined)
                self.targets.append(y_sample)
                self.indices.append(i)

        if len(self.features) > 0:
            self.features = np.array(self.features)
            self.targets = np.array(self.targets).reshape(-1, 1)
            self.indices = np.array(self.indices)
            self.scale_y = scale_y
            if scale_y:
                self.y_scaler = RobustScaler(quantile_range=(5.0, 95.0))
                self.targets_scaled = self.y_scaler.fit_transform(self.targets)
                self.targets_scaled = np.clip(self.targets_scaled, -10.0, 10.0)
            else:
                self.targets_scaled = self.targets
                self.y_scaler = None
        else:
            print("Warning: No valid samples found for trend dataset")
            self.features = np.zeros((1, 1, self.f * 2))
            self.targets = np.zeros((1, 1))
            self.targets_scaled = np.zeros((1, 1))
            self.indices = np.zeros(1)
            self.scale_y = scale_y
            self.y_scaler = None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.features[idx], dtype=torch.float32),
            torch.tensor(self.targets_scaled[idx], dtype=torch.float32).view(-1)
        )

    def inverse_transform_y(self, y_tensor: torch.Tensor) -> np.ndarray:
        if self.scale_y and self.y_scaler is not None:
            return self.y_scaler.inverse_transform(y_tensor.detach().cpu().numpy())
        else:
            return y_tensor.detach().cpu().numpy()

def train(model, criterion, optimizer, train_dataloader, scheduler=None):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_y in train_dataloader:
        if torch.isnan(batch_X).any() or torch.isnan(batch_y).any():
            batch_X = torch.nan_to_num(batch_X, nan=0.0)
            batch_y = torch.nan_to_num(batch_y, nan=0.0)
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        epoch_loss += loss.item()

    if scheduler is not None:
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(epoch_loss / len(train_dataloader))
        else:
            scheduler.step()
    return epoch_loss / len(train_dataloader)

def validate(model, val_dataloader, dataset):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch_X, batch_y in val_dataloader:
            if torch.isnan(batch_X).any() or torch.isnan(batch_y).any():
                batch_X = torch.nan_to_num(batch_X, nan=0.0)
                batch_y = torch.nan_to_num(batch_y, nan=0.0)
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            if torch.isnan(outputs).any():
                outputs = torch.nan_to_num(outputs, nan=0.0)
            all_targets.append(batch_y.cpu())
            all_preds.append(outputs.cpu())
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    all_preds = torch.nan_to_num(all_preds, nan=0.0)
    all_targets = torch.nan_to_num(all_targets, nan=0.0)
    np_preds = all_preds.numpy()
    np_targets = all_targets.numpy()
    mse = mean_squared_error(np_targets, np_preds)
    r2 = r2_score(np_targets, np_preds)
    try:
        corr = np.corrcoef(np_targets.flatten(), np_preds.flatten())[0, 1]
    except:
        corr = 0.0
    y_pred = dataset.inverse_transform_y(all_preds)
    y_true = dataset.inverse_transform_y(all_targets)
    true_mse = mean_squared_error(y_true, y_pred)
    true_r2 = r2_score(y_true, y_pred)

    return mse, r2, corr, true_mse, true_r2

def ultra_ensemble_predict(X_batch, ensemble_model, trend_model, weights=None):
    """Combines predictions from ensemble models"""
    if weights is None:
        weights = [0.6, 0.0, 0.4]

    X_batch = X_batch.to(device)

    ensemble_model.eval()
    with torch.no_grad():
        ensemble_pred = ensemble_model(X_batch).cpu()

    X_np = X_batch.cpu().numpy()
    n, t, f = X_np.shape
    trend_features = []

    for i in range(n):
        x_sample = X_np[i]
        if t > 1:
            x_pct_change = np.zeros((t-1, f))
            for j in range(t-1):
                denom = np.abs(x_sample[j]) + 1e-8
                x_pct_change[j] = (x_sample[j+1] - x_sample[j]) / denom
            x_pct_change = np.clip(x_pct_change, -10.0, 10.0)
            x_combined = np.column_stack([
                x_pct_change.reshape(-1),
                x_sample[:-1].reshape(-1)
            ])

            trend_features.append(x_combined)

    if trend_features:
        trend_features = torch.tensor(np.array(trend_features), dtype=torch.float32).to(device)
        trend_model.eval()
        with torch.no_grad():
            trend_pred = trend_model(trend_features).cpu()
    else:
        trend_pred = ensemble_pred

    final_pred = weights[0] * ensemble_pred + weights[2] * trend_pred

    return final_pred

def load_data():
    """Loads your data files"""
    features = np.load(f'/content/{model_type}_features.npy')
    targets = np.load(f'/content/{model_type}_target.npy')
    index = np.load(f'/content/{model_type}_index.npy')
    return features, targets, index

 r2, _, true_mse, true_r2 = validate(trend_model, trend_test_loader, trend_test_data)
            prdef reproduce_best_results():
    set_all_seeds(42)


    print(f"Loading data for {model_type} model...")
    features, targets, index = load_data()

    best_params = {
        'hidden_size': 256,
        'num_layers': 3,
        'dropout': 0.12443573657858692,
        'learning_rate': 0.009988439410223473,
        'batch_size': 16,
        'bidirectional': False,
        'weight_decay': 4.691096730777323e-06,
        'r2_weight': 0.5541537325865673,
        'loss_type': 'weighted'
    }

    hidden_size = best_params['hidden_size']
    num_layers = best_params['num_layers']
    dropout = best_params['dropout']
    learning_rate = best_params['learning_rate']
    batch_size = best_params['batch_size']
    bidirectional = best_params['bidirectional']
    weight_decay = best_params['weight_decay']
    r2_weight = best_params['r2_weight']
    mse_weight = 1.0 - r2_weight
    loss_type = best_params['loss_type']

    X_train, X_test, y_train, y_test, _, index_test = train_test_split(
        features,
        targets,
        index,
        test_size=0.2,
        shuffle=True,
        stratify=index[:, 1],
        random_state=42
    )

    train_data = TimeSeries(X_train, y_train, noise_level=0.0001)
    test_data = TimeSeries(X_test, y_test, noise_level=0)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)

    print("\nTraining ensemble model...")
    models = []
    for i in range(3):
        set_all_seeds(42 + i)
        h_size = int(hidden_size * (0.9 + 0.2 * np.random.random()))
        d_out = dropout * (0.8 + 0.4 * np.random.random())

        model = AttentionLSTM(
            input_size=features.shape[2],
            hidden_size=h_size,
            num_layers=num_layers,
            dropout=d_out,
            output_size=1,
            bidirectional=bidirectional
        ).to(device)

        models.append(model)

    set_all_seeds(42)
    ensemble_model = EnsembleModel(models, weights=[0.5, 0.3, 0.2]).to(device)

    if loss_type == "r2":
        criterion = R2Loss()
    else:
        criterion = WeightedR2MSELoss(r2_weight=r2_weight, mse_weight=mse_weight)

    optimizer = optim.AdamW(ensemble_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler1 = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=10, verbose=False, min_lr=1e-6)
    scheduler2 = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=learning_rate/20)

    num_epochs = 150
    for epoch in tqdm(range(num_epochs)):
        train_loss = train(ensemble_model, criterion, optimizer, train_loader)
        scheduler2.step()
        mse, r2, corr, true_mse, true_r2 = validate(ensemble_model, test_loader, test_data)
        scheduler1.step(train_loss)

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.7f}, "
                  f"Val MSE: {true_mse:.8f}, Val R²: {true_r2:.7f}, Val Corr: {corr:.7f}")

    _, _, _, ensemble_mse, ensemble_r2 = validate(ensemble_model, test_loader, test_data)
    print(f"Ensemble Model Results: MSE: {ensemble_mse:.8f}, R²: {ensemble_r2:.7f}")
    print("\nTraining trend model...")
    set_all_seeds(42)

    trend_train_data = TrendDataset(X_train, y_train)
    trend_test_data = TrendDataset(X_test, y_test)
    input_dim = trend_train_data.features.shape[1] * trend_train_data.features.shape[2]
    trend_train_loader = DataLoader(trend_train_data, batch_size=32, shuffle=True, num_workers=0)
    trend_test_loader = DataLoader(trend_test_data, batch_size=32, shuffle=False, num_workers=0)

    trend_model = TrendModel(
        input_size=input_dim,
        hidden_size=128,
        output_size=1,
        dropout=0.2
    ).to(device)

    trend_criterion = R2Loss()
    trend_optimizer = optim.AdamW(trend_model.parameters(), lr=0.001, weight_decay=1e-5)
    trend_scheduler = ReduceLROnPlateau(trend_optimizer, mode='min', factor=0.5, patience=10, verbose=False)

    trend_epochs = 60
    for epoch in tqdm(range(trend_epochs)):
        train_loss = train(trend_model, trend_criterion, trend_optimizer, trend_train_loader, trend_scheduler)

        if (epoch + 1) % 10 == 0:
            _,int(f"Trend Model - Epoch {epoch+1}/{trend_epochs}, Train Loss: {train_loss:.7f}, "
                  f"Val R²: {true_r2:.7f}, Val MSE: {true_mse:.8f}")

    _, _, _, trend_mse, trend_r2 = validate(trend_model, trend_test_loader, trend_test_data)
    print(f"Trend Model Results: MSE: {trend_mse:.8f}, R²: {trend_r2:.7f}")

    print("\nEvaluating ultra ensemble with fixed weights: [0.6, 0.0, 0.4]")
    ultra_weights = [0.6, 0.0, 0.4]

    all_preds = []
    all_targets = []

    for X_batch, y_batch in test_loader:
        pred = ultra_ensemble_predict(X_batch, ensemble_model, trend_model, ultra_weights)
        all_preds.append(pred)
        all_targets.append(y_batch)

    y_pred_tensor = torch.cat(all_preds)
    y_true_tensor = torch.cat(all_targets)

    y_pred = test_data.inverse_transform_y(y_pred_tensor)
    y_true = test_data.inverse_transform_y(y_true_tensor)

    ultra_mse = mean_squared_error(y_true, y_pred)
    ultra_r2 = r2_score(y_true, y_pred)
    ultra_corr = np.corrcoef(y_true.flatten(), y_pred.flatten())[0, 1]

    print("\nUltra Ensemble Final Results:")
    print(f"MSE: {ultra_mse:.8f}")
    print(f"R²: {ultra_r2:.7f}")
    print(f"Correlation: {ultra_corr:.4f}")

    save_path = f"/content/models/{model_type}_reproducible_model.pth"
    torch.save({
        'ensemble_model': ensemble_model.state_dict(),
        'trend_model': trend_model.state_dict(),
        'weights': ultra_weights,
        'params': best_params,
        'performance': {'r2': ultra_r2, 'mse': ultra_mse, 'corr': ultra_corr}
    }, save_path)

    print(f"Saved reproducible model to {save_path}")

    return {
        'ensemble_model': ensemble_model,
        'trend_model': trend_model,
        'weights': ultra_weights,
        'params': best_params,
        'performance': {'r2': ultra_r2, 'mse': ultra_mse, 'corr': ultra_corr}
    }

if __name__ == "__main__":
    model_result = reproduce_best_results()
    print("\nPROCESS COMPLETED")

Using device: cuda
Loading data for bladder model...


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning:

The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.




Training ensemble model...


  7%|▋         | 10/150 [00:09<02:08,  1.09it/s]

Epoch 10/150, Train Loss: -0.2601079, Val MSE: 0.00000019, Val R²: 0.3383859, Val Corr: 0.5918068


...

Trend Model - Epoch 40/60, Train Loss: -0.8715709, Val R²: 0.3491151, Val MSE: 0.00000019


 85%|████████▌ | 51/60 [00:07<00:01,  6.80it/s]

Trend Model - Epoch 50/60, Train Loss: -0.7570299, Val R²: 0.4151248, Val MSE: 0.00000017


100%|██████████| 60/60 [00:08<00:00,  7.24it/s]


Trend Model - Epoch 60/60, Train Loss: -0.8796713, Val R²: 0.3757032, Val MSE: 0.00000018
Trend Model Results: MSE: 0.00000018, R²: 0.3757032

Evaluating ultra ensemble with fixed weights: [0.6, 0.0, 0.4]

Ultra Ensemble Final Results:
MSE: 0.00000011
R²: 0.6277786
Correlation: 0.7925
Saved reproducible model to /content/models/bladder_reproducible_model.pth

PROCESS COMPLETED
